In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
df = pd.read_csv('Churn_Modeling_Mongodb.csv')
df.head()

,Unnamed: 0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,2,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
3,3,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0
4,4,11,15767821,Bearce,528,France,Male,31,6,102016.72,2,0,0,80181.12,0


In [3]:
df.drop(columns=['Unnamed: 0'],inplace=True)

## Data Preprocessing

In [4]:
# Drop irrevelant columns

df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [5]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
3,684,France,Male,27,2,134603.88,1,1,1,71725.73,0
4,528,France,Male,31,6,102016.72,2,0,0,80181.12,0


In [6]:
df.duplicated().sum()

0

In [7]:
# Encode categorical variable Gender

label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
3,684,France,1,27,2,134603.88,1,1,1,71725.73,0
4,528,France,1,31,6,102016.72,2,0,0,80181.12,0


In [8]:
# OneHot encode 'Geography'ArithmeticError

from sklearn.preprocessing import OneHotEncoder

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']])
geo_encoder =geo_encoder.toarray()

In [9]:
feat = onehot_encoder_geo.get_feature_names_out(['Geography'])

In [10]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=feat)
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [11]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
3,684,France,1,27,2,134603.88,1,1,1,71725.73,0
4,528,France,1,31,6,102016.72,2,0,0,80181.12,0


In [12]:
df.shape

(10000, 11)

In [13]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [14]:
geo_encoded_df.shape

(10000, 3)

In [15]:
df = pd.concat([df,geo_encoded_df],axis=1)

In [16]:
df.drop(columns=['Geography'],axis=1,inplace=True)

In [17]:
# save onehot and label encoder object

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [18]:
# Seperate dependent and independent variable

x = df.drop('Exited',axis=1)
y= df['Exited']

In [19]:
# split data into train test

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


In [20]:
# Scale the features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential  
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense

In [33]:
X_train[1].shape

(12,)

In [51]:
model = Sequential([
    Dense(64,activation='relu',input_shape=X_train[1].shape),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [37]:
import datetime

In [53]:

from tensorflow.keras.callbacks import TensorBoard

log_dir = "logs/fit"
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [45]:
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

In [54]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4533 - accuracy: 0.8087 - val_loss: 0.4017 - val_accuracy: 0.8230
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3854 - accuracy: 0.8444 - val_loss: 0.3661 - val_accuracy: 0.8480
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3568 - accuracy: 0.8566 - val_loss: 0.3462 - val_accuracy: 0.8540
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3475 - accuracy: 0.8599 - val_loss: 0.3385 - val_accuracy: 0.8585
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3422 - accuracy: 0.8608 - val_loss: 0.3352 - val_accuracy: 0.8585
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3384 - accuracy: 0.8611 - val_loss: 0.3386 - val_accuracy: 0.8570
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3343 - accuracy: 0.8627 - val_loss: 0.3343 - val_accuracy: 0.8585

In [47]:
model.save('model.h5')

c:\Users\fifty\anaconda3\envs\churn_project\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [55]:
# load TensorBoard Extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [58]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6008 (pid 24532), started 0:00:04 ago. (Use '!kill 24532' to kill it.)